# Usage estimation for Above Ground Biomass

<b>Table of Contents</b>

- Overview
- Setup
    - Get authentication token
- Query to estimate usage
    - Understanding the output

## Overview

In this section, we leverage the `dry run` functionality of the Geospatial APIs to estimate the usage in bytes for a V4 Above Ground Biomass query, based on the specified spatial and temporal ranges. The same feature can also be applied to calculate usage for datasets under V3 APIs as well. By leveraging this feature, users can get a better understanding of resource consumption before actually running the queries.


## Setup

Before starting, you have to install IBM EI Geospatial python SDK i.e ibmpairs and other modules needed to run this sample, mentioned in the below cell.

<b>Note:</b> To run this notebook seamlessly, you have to first configure your Geospatial APIs' credentials in a file named `secrets.ini` in the below format:

```
[EI]
api.api_key = <Your Geospatial APIs API key>
api.tenant_id = <Your Geospatial APIs Tenant ID>
api.org_id = <Your Geospatial APIs Org ID>

```
Keep the secrets.ini file at an appropriate relative location of this notebook. For example, as specified in the config below.

```config.read('../../../auth/secrets.ini')```

In [ ]:
%pip install ibmpairs
%pip install configparser

In [35]:
import ibmpairs.client as client
import ibmpairs.query as query
import configparser

### Get authentication token

Now, let's create a client object using the API_KEY, TENANT_ID (or CLIENT_ID) and ORG_ID to create an authenticated HTTP client Authentication token using 'ibmpairs.client' module, to use it in the subsequent steps.

In [36]:
config = configparser.RawConfigParser()
config.read('../../../auth/secrets.ini')

EI_ORG_ID     = config.get('EI', 'api.org_id') 
EI_TENANT_ID  = config.get('EI', 'api.tenant_id') 
EI_APIKEY     = config.get('EI', 'api.api_key')

EI_client_v4 = client.get_client(org_id    = EI_ORG_ID,
                                  tenant_id = EI_TENANT_ID,
                                  api_key     = EI_APIKEY,
                                  version   = 4
                                )

2025-02-13 17:25:37 - paw - INFO - The client authentication method is assumed to be OAuth2.
2025-02-13 17:25:37 - paw - INFO - Legacy Environment is False
2025-02-13 17:25:37 - paw - INFO - The authentication api key type is assumed to be IBM EIS, because the api key prefix 'PHX' is present.
2025-02-13 17:25:40 - paw - INFO - Authentication success.
2025-02-13 17:25:40 - paw - INFO - HOST: https://api.ibm.com/geospatial/run/na/core/v4


## Query to estimate usage

The following code snippet submits the query for the AGB Data Layer <b><i>above-ground-biomass</i></b> for an area in Brazil spanning approximately 10,000 hectares for the year 2023 using Geospatial APIs V4 query API with the `dryrun` enabled to estimate resource usage.

We will go ahead and submit the query and observe the output.

In [39]:
above_ground_biomass_query_object = query.submit({ 
                          "name": "Above Ground Biomass - usage calculation",
                          "dryRun": True,
                          "spatial": {
                              "geojson": {
                                  "type": "Feature",
                                  "geometry": {
                                      "type":"MultiPolygon",
                                       "coordinates": [
                                           [
                                                [[-60.21655835189182 , -2.8902202011994365], 
                                                [-60.221233764112455, -2.7948417918984165], 
                                                [-60.13520617925271 , -2.7864260499012676], 
                                                [-60.1211799425908  , -2.882739541646415 ], 
                                                [-60.21655835189182 , -2.8902202011994365]] 
                                           ]
                                       ]
                                  }   
                              } 
                          },
                          "temporal": {
                              "years": [2023] 
                          },
                          "layers": [
                              {
                                  "datalayer": "above-ground-biomass" 
                              }
                            ]
                          },EI_client_v4)

print(above_ground_biomass_query_object.submit_response)

2025-02-13 17:34:16 - paw - INFO - TASK: submit STARTING.
2025-02-13 17:34:19 - paw - INFO - TASK: submit COMPLETED.
{
    "info": {
        "count": 0,
        "interactive_max_count": 0,
        "max_count": 0,
        "max_usage": 6442450.944,
        "usage": 815052,
        "usage_unit": "bytes"
    }
}


### Understanding the output
The API response provides an estimated usage of 815,052 bytes per API call when querying the Above Ground Biomass data for a 10,000-hectare plot in Brazil for the year 2023. Please note that the usage estimate increases with the addition of historical years and expansion of spatial area within the query. However, the estimate remains unchanged when future years are added, as it is calculated based on the historical years available in the query for which data exists.

Similarly, usage can be calculated for different spatial and temporal ranges of interest, ensuring resource estimates tailored to specific query parameters.